# REWOO Pattern Implementation with Strands Multiagent Graph

## Overview

This implementation demonstrates the **REWOO (Reasoning Without Observation)** pattern using the Strands multiagent graph framework. REWOO separates reasoning (planning) from acting (tool execution), enabling more reliable and debuggable agent workflows. It is based on https://github.com/langchain-ai/langgraph/blob/main/docs/docs/tutorials/rewoo/rewoo.ipynb

## Architecture

The system consists of three specialized agents connected in a sequential graph:

```
User Query → [Planner] → [Worker] → [Solver] → Final Response
```

### 1. Planner Agent
- **Purpose**: Generates structured execution plans without executing tools
- **Input**: User query (e.g., "Check my flight and change to earlier time")
- **Output**: Structured plan with numbered steps:
  ```
  Plan 1: Retrieve flight information
  #E1 = fetch_user_flight_information[passenger_id="3442 587242"]
  
  Plan 2: Search for earlier flights
  #E2 = search_flights[departure_airport=departure_airport, ...]
  ```
- **Implementation**: Custom agent class that yields proper `AgentResult` objects

### 2. Worker Agent  
- **Purpose**: Executes the plan by calling appropriate tools deterministically
- **Input**: Plan from planner + original user query
- **Process**: 
  - Parses plan steps using regex
  - Executes tools in sequence (#E1 → #E2 → #E3)
  - Resolves parameter dependencies between steps
- **Output**: Evidence dictionary with execution results
- **Implementation**: Custom agent class with `execute_flight_plan` tool

### 3. Solver Agent
- **Purpose**: Synthesizes evidence into user-friendly responses
- **Input**: Original query + execution evidence from worker
- **Process**: Combines evidence with user query to generate final answer
- **Output**: Natural language response to user
- **Implementation**: Custom agent class with `solve_flight_query` tool

## Key Technical Details

### Custom Agent Classes
Due to multiagent graph requirements, each agent extends the base `Agent` class with custom `stream_async()` methods that properly yield `AgentResult` objects:

```python
class PlannerAgent(Agent):
    async def stream_async(self, prompt: str):
        # Generate plan using tool
        plan_result = self.tool.generate_flight_plan(user_query=prompt)
        
        # Create proper AgentResult
        message = Message(content=[{"text": str(plan_result)}])
        agent_result = AgentResult(
            stop_reason="end_turn",

                        metrics=EventLoopMetrics(),
            state=None
        )
        
        yield {"result": agent_result}
```

### Information Flow
1. **Planner Input**: Raw user query
2. **Worker Input**: Multiagent graph automatically combines:
   - Original task
   - Planner output (the plan)
3. **Solver Input**: Multiagent graph automatically combines:
   - Original task  
   - Worker output (execution evidence)

## Usage

```python
# Create the multiagent graph
rewoo_graph = create_rewoo_graph()

# Execute with user query
result = await rewoo_graph.execute_async(
    "Hi what time is my flight, my passenger id is '3442 587242'. Change my flight to earlier time."
)

# Access results from each agent
planner_result = result.results['planner']
worker_result = result.results['worker'] 
solver_result = result.results['solver']
```

This implementation showcases how the Strands multiagent graph framework can orchestrate complex multi-step reasoning workflows while maintaining clear separation between planning, execution, and response synthesis.

In [ ]:
!pip3 install -r ./requirements.txt --quiet #--upgrade
!pip3 install strands-agents==v1.6.0 strands-agents-tools==v0.2.5

#!pip3 install strands-agents[a2a] strands-agents-tools --quiet


In [81]:
import time
import boto3
import ipywidgets as widgets
import uuid
import pandas as pd
import numpy as np
import os
import shutil
import sqlite3
import functools
import requests
import pytz
import warnings
from IPython.display import Image, display
from botocore.config import Config
from typing import Annotated, Literal, Optional, Union
from typing_extensions import TypedDict
from bs4 import BeautifulSoup
from datetime import date, datetime
from typing import List, Dict, Any
import re
import json
import base64


from strands import Agent
from strands import tool
from strands.models import BedrockModel
from strands.agent.conversation_manager import SlidingWindowConversationManager

from strands.multiagent.graph import GraphBuilder
from strands.agent import AgentResult
from strands.types.content import Message
from strands.types.streaming import StopReason
from strands.telemetry.metrics import EventLoopMetrics
from strands.telemetry.config import StrandsTelemetry
import logging

from rewoo_helper_funcs import *
from bedrock_helper import get_bedrock_response, get_claude_response, get_claude_response_text

### Use Strands 


In [82]:
#Clients
# Create BedrockModel with specified region
region="us-east-1"
bedrock_model_taubench = BedrockModel(region_name= region)

#Conversation Manager
conv_manager = SlidingWindowConversationManager(window_size=10)

#setup logging
# Disable all logging except critical errors
logging.basicConfig(level=logging.CRITICAL)

# Silence specific noisy loggers completely
for logger_name in ["strands", "graph", "event_loop", "registry", "sliding_window_conversation_manager", "bedrock", "streaming"]:
    logging.getLogger(logger_name).setLevel(logging.CRITICAL)


## Get MAbench and Taubench tools

In [83]:
# Libraries

import sys
sys.path.append('../data/ma-bench/')
sys.path.append('../data/tau-bench/')

from mabench.environments.airline.tools.book_reservation import book_reservation
from mabench.environments.airline.tools.calculate import calculate
from mabench.environments.airline.tools.cancel_reservation import cancel_reservation
from mabench.environments.airline.tools.get_reservation_details import get_reservation_details
from mabench.environments.airline.tools.get_user_details import get_user_details
from mabench.environments.airline.tools.list_all_airports import list_all_airports
from mabench.environments.airline.tools.search_direct_flight import search_direct_flight
from mabench.environments.airline.tools.search_onestop_flight import search_onestop_flight
from mabench.environments.airline.tools.send_certificate import send_certificate
from mabench.environments.airline.tools.think import think
from mabench.environments.airline.tools.transfer_to_human_agents import transfer_to_human_agents
from mabench.environments.airline.tools.update_reservation_baggages import update_reservation_baggages
from mabench.environments.airline.tools.update_reservation_flights import update_reservation_flights
from mabench.environments.airline.tools.update_reservation_passengers import update_reservation_passengers

domain = "airline"

# from tau_bench.envs.tool import Tool
# from tau_bench.envs.airline.tools import *
from tau_bench.envs.airline.data import *
from tau_bench.envs.airline.tasks import *
from tau_bench.envs.airline.wiki import WIKI

### REWOO Orchestration

### PLANNER : Receives user query and makes the plan
"""
PLANNER AGENT - REWOO Step 1: Reasoning
Generates structured execution plans without tool execution. Takes user queries and creates 
step-by-step plans in #E1, #E2, #E3 format specifying which tools to call and with what parameters.
Custom stream_async() ensures proper AgentResult yielding for multiagent graph compatibility. (in last cell of notebook)
"""

In [84]:
def direct_llm_call(prompt):
    max_tokens = 2048
    temp = 0
    topP = 1
    response = get_claude_response(user_message=prompt,
                                    model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0", 
                                    max_tokens=max_tokens, 
                                   temp=temp)                
    answer = get_claude_response_text(response)
    return answer


In [85]:
@tool
def generate_flight_plan(user_query: str) -> str:
    """Generate a structured flight plan for the given user query"""
    print(f"inside flight plan tool \n")
    # planner prompt
    planner_prompt = """
# PLANNING ONLY ASSISTANT - DO NOT EXECUTE

Your ONLY job is to write a plan using the exact format below. You MUST NOT try to execute the plan or have any other interactions.

## Available Flight Tools
* calculate[expression]
* get_reservation_details[reservation_id]
* update_reservation_flights[reservation_id, cabin, flights, payment_id]
* search_onestop_flight[origin, destination, date]
* send_certificate[user_id, amount]
* cancel_reservation[reservation_id]
* search_direct_flight[origin, destination, date]
* get_user_details[user_id]
* list_all_airports[]
* book_reservation[user_id, origin, destination, flight_type, cabin, flights, passengers, payment_methods, total_baggages, nonfree_baggages, insurance]
* think[thought]
* transfer_to_human_agents[summary]
* update_reservation_passengers[reservation_id, passengers]
* update_reservation_baggages[reservation_id, total_baggages, nonfree_baggages, payment_id]
* book_reservation[user_id, origin, destination, flight_type, cabin, flights, passengers, payment_methods, total_baggages, nonfree_baggages, insurance]
* cancel_reservation[reservation_id]
* calculate[expression]

## REPEAT Syntax and Usage
When multiple iterations of the same steps are needed, use this format:

1. First, use think tool to analyze and count items to process
2. Then, use another think tool to plan iteration details
3. Finally, use REPEAT block with the count from previous steps

REPEAT(count_from_previous_step) {
    tool1[parameters]
    tool2[parameters]
    ...
}

Available variables in REPEAT blocks:
- CURRENT_ITERATION (0-based index)
- CURRENT_ITEM (from list being processed)
- Other variables extracted from previous steps

Use REPEAT blocks when:
- Processing multiple reservations
- Applying multiple certificates
- Handling multiple passengers
- Any task that requires the same steps multiple times

Note: Evidence numbers inside REPEAT will be expanded sequentially


## Required Format - USE EXACTLY THIS:

Plan 1: [Description]
#E1 = [tool_name][parameters]

Plan 2: [Description]
#E2 = [tool_name][parameters]

## Examples:


Example 1 : "Can you put me on an earlier flight? My reservation ID is 'CD789012'"
Plan 1: Retrieve the current reservation details
#E1 = get_reservation_details[reservation_id="CD789012"]

Plan 2: Search for earlier direct flights based on the origin, destination and date from #E1
#E2 = search_direct_flight[origin=origin_airport_code, destination=destination_airport_code, date=travel_date]

Plan 3: Update the reservation with the earlier flight found in #E2 and use details from #E1 and useer query as necessary
#E3 = update_reservation_flights[reservation_id="CD789012", cabin=cabin_class, flights=selected_flights, payment_id=payment_info]

Example 2 : "My user id is mia_li_3668. I want to fly from New York to Seattle on May 20 (one way). I do not want to fly before 11am EST. I want to fly in economy. I prefer direct flights but one stopover is also fine. If there are multiple options, I prefer the one with the lowest price. I have 3 baggages. I do not want insurance. I want to use my two certificates to pay. If only one certificate can be used, I prefer using the larger one, and pay the rest with my 7447 card"
Plan 1: Get user details to check available certificates
#E1 = get_user_details[user_id="mia_li_3668"]

Plan 2: Get list of airports to find the airport codes for New York and Seattle
#E2 = list_all_airports[]

Plan 3: Search for direct flights using airport codes from #E2 and date from given user question
#E3 = search_direct_flight[origin=origin_airport_code, destination=destination_airport_code, date=travel_date]

Plan 4: If no suitable direct flights, search for one-stop flights using using airport codes from #E2 and date from given user question
#E4 = search_onestop_flight[origin=origin_airport_code, destination=destination_airport_code, date=travel_date]

Plan 5: Return selected flights from #E4  and #E3 


Example 3 : "I have a booking number TR7845. I need to update my daughter's name from Emma Wilson to Emma Thompson as she recently got married. I'm Jennifer Wilson, ID: TW5432P891."

Plan 1: Retrieve the current reservation details
#E1 = get_reservation_details[reservation_id="TR7845"]

Plan 2: Verify user identity and authorization
#E2 = get_user_details[user_id="TW5432P891"]

Plan 3: Think about the passenger name changes needed
#E3 = think["Analyze the user query and reservation details:

Identify the passenger whose name needs to be changed: Emma Wilson
New name for this passenger: Emma Thompson
Keep all other passengers unchanged
Preserve existing passenger details (DOB, etc) from reservation
Create an updated passenger list with the name change"]

Plan 4: Update the reservation with modified passenger information
#E4 = update_reservation_passengers[reservation_id="TR7845", passengers=[
{"first_name": "Jennifer", "last_name": "Wilson", "dob": extracted_dob_jennifer},
{"first_name": "Emma", "last_name": "Thompson", "dob": extracted_dob_emma}
]]

Example 4 : "Hi, my name is Jordan Smith (customer ID: ZX7890Y123). I have a reservation with booking code LM5678 for a flight from Chicago to Miami on June 15. 
I need to add my son, Alex Smith, to the reservation and include an extra bag for him. Can you help me with that?"

Plan 1: Retrieve the current reservation details
#E1 = get_reservation_details[reservation_id="LM5678"]

Plan 2: Verify user identity and authorization
#E2 = get_user_details[user_id="ZX7890Y123"]

Plan 3: Think about required passenger updates
#E3 = think["Analyze current reservation and requested changes:

Get existing passenger list from #E1
New passenger to add: Alex Smith (son)
Need to preserve all existing passenger details
Create updated passenger list that includes both existing and new passengers"]

Plan 4: Update the reservation with complete passenger list
#E4 = update_reservation_passengers[reservation_id="LM5678", passengers=[
extract_existing_passengers_from_E1,
{"first_name": "Alex", "last_name": "Smith", "type": "child"}
]]

Plan 5: Think about baggage update
#E5 = think["Calculate baggage updates:

Get current total_baggages from #E1
Add one extra bag for new passenger
Determine if extra bag is free or paid based on cabin class"]

Plan 6: Update the baggage count
#E6 = update_reservation_baggages[
reservation_id="LM5678",
total_baggages=current_total_plus_one,
nonfree_baggages=current_nonfree_plus_one,
payment_id=payment_from_context
]

Example 5: "My user id is ABC123. I want to downgrade all my business flights to economy for my reservations. Please calculate total savings."

Plan 1: Get user details to retrieve all reservations
#E1 = get_user_details[user_id="ABC123"]

Plan 2: REPEAT(length_of_reservations_from_#E1) {
    get_reservation_details[reservation_id=CURRENT_RESERVATION_ID]    
    calculate["current_savings = business_fare - economy_fare"]
    calculate["total_savings += current_savings"]
    update_reservation_flights[reservation_id=CURRENT_RESERVATION_ID, cabin="economy", flights=CURRENT_FLIGHTS, payment_id=CURRENT_PAYMENT]
}


## IMPORTANT: 
1. ONLY write the plan - nothing else
2. Do NOT add any explanations or clarifications
3. Do NOT attempt to execute any actions
4. Follow the format exactly as shown
5. Use 'think' tool only when needed like name change.


<policy>
{policy}
</policy>
"""

    planning_llm = Agent(
        model=bedrock_model_taubench,
        system_prompt=planner_prompt.replace("{policy}", WIKI)
    )
    plan = planning_llm(user_query)
    
    return str(plan)



### EXECUTOR: receives the plan and executes it
"""
WORKER AGENT - REWOO Step 2: Observation
Executes the structured plan from Planner by calling flight tools deterministically.
Parses plan steps, resolves parameter dependencies between steps, and returns evidence dictionary.
Uses execute_flight_plan tool to orchestrate sequential tool execution (#E1 → #E2 → #E3).
"""

In [112]:
import json
from typing import Any, Dict, List, Optional, Union



REPEAT_ANALYSIS_PROMPT = """Given this JSON response from a previous step:
{json_data}

And this REPEAT condition:
{repeat_condition}

Task:
1. Determine which list in the JSON needs to be counted
2. Count the number of items in that list
3. Extract all items from that list

Return ONLY in this exact format:
Count: [number]
List: [comma-separated items]"""

@tool
def execute_flight_plan(plan: str) -> str:
    """Execute plan with fully dynamic parameter resolution"""

    user_query=  extract_original_task(plan)
    
    steps = []
    regular_step_pattern = r'#E(\d+)\s*=\s*(\w+)\[([^\]]*)\]'
    repeat_block_pattern = r'#E(\d+)\s*=\s*REPEAT\(([^)]+)\)\s*\{([^}]+)\}'
    
    # First find all regular steps
    regular_steps = re.finditer(regular_step_pattern, plan)
    for match in regular_steps:
        steps.append(('regular', match.group(1), match.group(2), match.group(3)))
        
    # Then find REPEAT blocks
    repeat_blocks = re.finditer(repeat_block_pattern, plan)
    for match in repeat_blocks:
        steps.append(('repeat', match.group(1), match.group(2), match.group(3)))
    
    # Sort steps by evidence number
    steps.sort(key=lambda x: int(x[1]))
    
    all_evidence = {}  # This will store ALL evidences
    current_evidence_num = 1
    
    for step_type, evidence_id, *rest in steps:
        if step_type == 'regular':
            tool_name, args_str = rest
            new_evidence = execute_single_step(evidence_id, tool_name, args_str, all_evidence.copy(), user_query)
            # Merge new evidence into all_evidence
            all_evidence.update(new_evidence)
            
        else:
            repeat_condition, block_content = rest            
            repeat_evidence = handle_repeat_block(evidence_id, repeat_condition, block_content, all_evidence.copy(), user_query)
            # Merge repeat evidence into all_evidence
            all_evidence.update(repeat_evidence)
            
    return str(all_evidence) #all_evidence

def handle_repeat_block(evidence_id: str, repeat_condition: str, block_content: str, evidence: dict, user_query: str):
    """Handle execution of a REPEAT block"""
    # Extract source evidence number
    print("inside handle repeat block \n")
    repeat_evidence = {}  # Store evidence from repeat block
   
    # Use LLM to analyze JSON and repeat condition
    system_prompt = REPEAT_ANALYSIS_PROMPT.format(
        json_data=evidence,
        repeat_condition=repeat_condition  # e.g., "number_of_reservations_from_#E1"
    )
    llm_response = direct_llm_call(system_prompt)    
    count_match = re.search(r'Count:\s*(\d+)', llm_response)
    list_match = re.search(r'List:\s*([\w\d, ]+)', llm_response)  # More flexible pattern

    if not count_match:
        raise ValueError(f"Could not find count in LLM response: {llm_response}")
    
    count = int(count_match.group(1))
    
    if not list_match:
        # Fallback: try to extract items between commas after "List:"
        list_start = llm_response.find("List:") + 5
        items_text = llm_response[list_start:].strip()
        items = [item.strip() for item in items_text.split(',')]
    else:
        items = [item.strip() for item in list_match.group(1).split(',')]

    # Execute block for each item
    current_evidence_num = int(evidence_id)
    for i, item in enumerate(items):
        print(f"\nDEBUG: REPEAT iteration {i+1}/{count}")
        
        # Parse and execute each step in the block
        step_pattern = r'(\w+)\[([^\]]*)\]'
        steps = re.findall(step_pattern, block_content)
        
        for tool_name, args_str in steps:
            # Replace placeholders
            print(f" tool_name {tool_name}  args_str {args_str} \n")
            processed_args = args_str.replace('CURRENT_RESERVATION_ID', item)
            processed_args = processed_args.replace('CURRENT_ITERATION', str(i))
            
            step_evidence = execute_single_step(str(current_evidence_num), tool_name, processed_args, evidence, user_query)
            repeat_evidence.update(step_evidence)
            current_evidence_num += 1
    
    return repeat_evidence

def execute_single_step(evidence_id: str, tool_name: str, args_str: str, evidence: dict, user_query: str):
    """Execute a single step and update evidence"""
    print(f"\nDEBUG: Processing step #E{evidence_id}")
    print(f"\nDEBUG: Tool name: {tool_name}")
    
    step_evidence = {}  # Store evidence for this step only
    # Parse keyword arguments
    kwargs = {}
    if args_str.strip():
        for arg_pair in args_str.split(','):
            arg_pair = arg_pair.strip()
            if '=' in arg_pair:
                key, value = arg_pair.split('=', 1)
                key = key.strip()
                value = value.strip().strip('"\'')
                kwargs[key] = value
    
    print(f"DEBUG: Final kwargs: {kwargs}")
    # Form the context        
    items = list(evidence.items()) 
    evidence_context = "\n".join([f"{k}: {v['results']}" for k, v in evidence.items()]) #currently combning all evidences for all steps prior to the current step
    # context_dict keeps structured results per step key (e.g., "#E2": {...} or list/str/int)
    context_dict= items    
    context = f"Evidence Context {evidence_context}\n\nUser Query: {user_query}"
   
    # Resolve arguments from context if needed
    if 'think' not in tool_name:           
        # LLM based argument resolution more latency but accurate
        kwargs = resolve_arguments(tool_name, kwargs, evidence, context, bedrock_model_taubench) 
        
        # Non-LLM based argument resolution i.e regex matching fromj dictionary 
        #kwargs=sanitize_kwargs(kwargs)
        #kwargs= resolve_kwargs_from_dict(kwargs, context_dict) #find from dict
       
    print(f"New kwargs {kwargs}")
    # Execute tool dynamically
    try:
        tool_func = getattr(worker_agent.tool, tool_name)
        
        if 'think' in tool_name:
            result = f" <think> {str(args_str)} </think>"
        elif 'calculate' in tool_name:
            calculate_prompt="""From the given <user_query>  and <evidence>  find the values that can be used for the <calculator_kwargs> that will be 
            passed to calculate tool. You must only return a math expression  to calculate, such as '2 + 2' which can be used by the 'calculate' tool. 
            You must only return the mathematical expression between 2 quotation marks.
            <user_query>
            {user_query}
            </user_query>
            <evidence>
            {evidence_context}
            </evidence>
            <calculator_kwargs>
            {args_str}
            </calculator_kwargs>
            """
            calculate_kwargs=direct_llm_call(system_prompt=calculate_prompt) 
            result = tool_func(calculate_kwargs)
            print(f"answer from calculate {calculate_kwargs}  {result} \n")
        else:
            if kwargs:
                print(f"DEBUG: Calling {tool_name} with kwargs: {kwargs}")
                result = tool_func(**kwargs)
            else:
                print(f"DEBUG: Calling {tool_name} with no args")
                result = tool_func()
        
        print(f"DEBUG: Tool result: {result}")
       
        # Store result
        if isinstance(result, dict) and 'content' in result:
            result_data = result['content'][0]['text']
        else:
            result_data = str(result)
        
        step_evidence[f'#E{evidence_id}'] = {
            'evidence_id': f'#E{evidence_id}',
            'description': f"Execute {tool_name} with {kwargs if kwargs else 'no parameters'}",
            'results': result_data
        }
            
    except Exception as e:
        step_evidence[f'#E{evidence_id}'] = {
            'evidence_id': f'#E{evidence_id}',
            'description': f"Execute {tool_name} with {kwargs if kwargs else 'no parameters'}",
            'results': f"Error: {str(e)}"
        }
    
    return step_evidence

def get_last_evidence_num(plan_section: str) -> int:
    """Get the last evidence number used in the plan section"""
    evidence_nums = re.findall(r'#E(\d+)', plan_section)
    return int(evidence_nums[-1]) if evidence_nums else 0

    

    


### SOLVER: Receives full plan and the responses of individual tool calls and prepares final response which is given to the user
"""
SOLVER AGENT - REWOO Step 3: Synthesis  
Combines execution evidence from Worker with original user query to generate final response.
Receives structured evidence dictionary and synthesizes it into natural language answer.
Uses solve_flight_query tool to create user-friendly responses from technical execution results.
"""

In [113]:
import ast


solve_prompt = """Solve the following task or problem. To solve the problem, we have made step-by-step Plan and retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer directly with no extra words.

Task: {task}
Response:"""


@tool
def solve_flight_query(user_query: str, planner_response: str, evidence_str: str) -> str:
    """
    Solve user query using structured evidence from worker execution
    """
   
    # Parse evidence string to dictionary
    try:        
        evidence_dict = ast.literal_eval(evidence_str)       
    except Exception as e:
        print(f"DEBUG: Failed to parse evidence_str: {e}")
        evidence_dict = {}
    
    # Build plan string from evidence
    plan = ""
    print(f"\nDEBUG: Building plan from evidence_dict with {len(evidence_dict)} items")
    for evidence_id, evidence_data in evidence_dict.items():
        
        if isinstance(evidence_data, dict):
            description = evidence_data.get('description', '')
            results = evidence_data.get('results', '')
            plan += f"Plan: {description}\n{evidence_id} = {results}\n\n"
            
    
    print(f"DEBUG: Final plan built")
    
    try:
        
        formatted_prompt = solve_prompt.format(plan=plan, task=user_query)        
        solved_answer = direct_llm_call(formatted_prompt)       
        result = str(solved_answer)
        return result
        
    except Exception as e:
        print(f"DEBUG: Exception in solve_flight_query: {e}")
        print(f"DEBUG: Exception type: {type(e)}")
        raise

## Make the REWOO strands graph

In [114]:
# Build rewoo graph

class PlannerAgent(Agent):
    async def stream_async(self, prompt: str):
    
        # NOTE: stream_async must accept **kwargs; Graph/Swarm pass callback_handler & more.
        print(f"DEBUG: PLANNER AGENT CALLED\n")

        # Call the tool and get result        
        prompt=normalize_prompt(prompt)        
        plan_result = self.tool.generate_flight_plan(user_query=prompt)
        
        # Create  AgentResult object with required parameters
        message = Message(content=[{"text": str(plan_result)}])
        metrics = EventLoopMetrics()
        print("DEBUG: PLANNER AGENT RESULT: \n", json.dumps(message), "\n")
        agent_result = AgentResult(
            stop_reason="end_turn",
            message=message,
            metrics=metrics,
            state=None
        )
        
        # Yield the result event that multiagent graph expects
        yield {"result": agent_result}

# Use Custom planner
planner_agent = PlannerAgent(
    model=bedrock_model_taubench,
    tools=[generate_flight_plan],
    name="planner"
)

class WorkerAgent(Agent):
    async def stream_async(self, prompt: str):
        # Call the execute_flight_plan tool with the plan from planner
        print(f"\n DEBUG: WORKER AGENT CALLED TO EXECUTE PLAN \n")
        prompt=normalize_prompt(prompt)
        
        evidence_result = self.tool.execute_flight_plan(plan=prompt)#add user query argument
        
        # Create AgentResult object
        message = Message(content=[{"text": str(evidence_result)}])
        print("DEBUG: WORKER AGENT RESULT: \n", json.dumps(message), "\n")
        metrics = EventLoopMetrics()
        
        agent_result = AgentResult(
            stop_reason="end_turn",
            message=message,
            metrics=metrics,
            state=None
        )
        
        # Yield the result event that multiagent graph expects
        yield {"result": agent_result}

# Use the custom worker
worker_agent = WorkerAgent(
    model=bedrock_model_taubench,
    tools=[
        book_reservation,
        calculate,
        cancel_reservation,
        get_reservation_details,
        get_user_details,
        list_all_airports,
        search_direct_flight,
        search_onestop_flight,
        send_certificate,
        think,
        transfer_to_human_agents,
        update_reservation_baggages,
        update_reservation_flights,
        update_reservation_passengers,
        execute_flight_plan
    ],
    name="worker"
)

class SolverAgent(Agent):
    async def stream_async(self, prompt: str):
        # Extract plan and evidence from the combined input
        # The prompt will contain both original task and worker results
        print(f"DEBUG: SOLVER AGENT CALLED TO FORM FINAL ANSWER FROM EXECUTED PLAN\n")
        prompt=normalize_prompt(prompt)
        
        original_task, evidence_str = extract_task_and_plans(prompt)
        print(f"ORIGINAL_TASK {original_task}\n")
        print(f"EVIDENCE STRING {evidence_str}\n")
        # Call solve_flight_query tool
        final_answer = self.tool.solve_flight_query(
            user_query=original_task,
            planner_response="",  # Not needed for solver
            evidence_str=str(evidence_str)
        )
        
        # Create AgentResult object
        message = Message(content=[{"text": str(final_answer)}])
        print("DEBUG: SOLVER AGENT RESULT: \n", json.dumps(message), "\n")
        metrics = EventLoopMetrics() # check how to get the eventloopmetrics
        
        agent_result = AgentResult(
            stop_reason="end_turn",
            message=message,
            metrics=metrics,
            state=None
        )
        
        yield {"result": agent_result}


# Use the custom solver
solver_agent = SolverAgent(
    model=bedrock_model_taubench,
    tools=[solve_flight_query],
    name="solver"
)


# Finally create the graph with the 3 agent nodes
def create_rewoo_graph():   
    builder = GraphBuilder()    
    # Add the three agents
    builder.add_node(planner_agent, "planner")
    builder.add_node(worker_agent, "worker")
    builder.add_node(solver_agent, "solver")
    
    # Sequential flow: planner -> worker -> solver
    builder.add_edge("planner", "worker")
    builder.add_edge("worker", "solver")
    
    builder.set_entry_point("planner")
    return builder.build()



In [ ]:
### Load the taubench dataset
output_path = os.path.join("..", "data", "tau-bench", "tau_bench", "envs", f"{domain}", "tasks_singleturn.json")
with open(output_path, "r") as file:
    tasks = json.load(file)
print(len(tasks))

In [116]:
# Create and execute
import json
import ast
import time


# previous without getting metrics
def test_rewoo_graph(question_id):
    task = tasks[question_id]
    #print(task)

    user_query = task["question"]    
    rewoo_graph = create_rewoo_graph()    
    start=time.time()
    #result = await rewoo_graph.invoke_async(user_query)
    result = rewoo_graph(user_query)    
    exec_time=time.time()-start
    print("=== REWOO Multiagent Graph Results ===")
    print(f"Graph execution time: {exec_time}")
    
    print(f"Status: {result.status}")
    print(f"Total nodes: {result.total_nodes}")
    print(f"Completed nodes: {result.completed_nodes}")
    filename = f"./output/rewoo_response_{question_id}.txt"
    
    try:
       
        with open(filename, "w", encoding="utf-8") as f:
            # Write execution summary
            f.write("=== REWOO Multiagent Graph Results ===\n")
            f.write(f"Status: {result.status}\n")
            f.write(f"Total nodes: {result.total_nodes}\n")
            f.write(f"Completed nodes: {result.completed_nodes}\n\n")
          
            # Write each node's result
            for node_id, node_result in result.results.items():
                print(f"\n--- {node_id.upper()} ---")
                
                # Write node separator
                f.write(f"\n{'='*50}\n")
                f.write(f"--- {node_id.upper()} ---\n")
                f.write(f"{'='*50}\n")
                
                try:
                    if hasattr(node_result.result, 'content'):
                        content = node_result.result.content
                        print(content)
                        f.write(str(content) + "\n")
                    else:
                        result_text = extract_text_from_response(str(node_result.result))
                        print(result_text)
                        f.write(result_text + "\n")
                except Exception as e:
                    error_msg = f"Error processing {node_id}: {str(e)}"
                    print(error_msg)
                    f.write(error_msg + "\n")
                
                f.write("\n")  # Add blank line between nodes
                
                 
    except Exception as e:
        print(f"Error writing to file {filename}: {str(e)}")
    

### Test with question_id

In [ ]:
question_id = 43 #20, #48

test_rewoo_graph(question_id)
